In [17]:
import ee
import geemap as emap
import os
import ee.mapclient

In [2]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

# Initialize Basemap

In [3]:
Map=emap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

# 1. Read shapefile area 

In [4]:
# Load point shapefile and HN area
HN=ee.FeatureCollection("users/havantuyen/Hanoi_Extent")
Map.centerObject(HN,9)
Map.addLayer(ee.Image().paint(HN,0,1.5),{"palette":["black"]},"HN_Map") # Colour = 0 or set color="red", and width =1.5

In [5]:
# Read Sentinel-2 Data and clip the HN
Sentinel2=ee.ImageCollection("COPERNICUS/S2").filterDate("2018-08-01","2018-11-30").filterBounds(HN)\
.filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",10).median().clip(HN)

# Map.addLayer(Sentinel2,{"bands":["B4","B3","B2"],"min":536.86,"max":3160},"Sentinel2")

## 1.1. Convert Sentinel-2 to TOA Reflectance 

In [76]:
TOA_S2=Sentinel2.divide(ee.Number(10000))
# Map.addLayer(TOA_S2,{"bands":["B4","B3","B2"],"min":0.08,"max":0.28},"Sentinel2_TOA")

# 2. Rainfall and Temperature Data

## 2.1. Precipitation

### 2.1.1. Obtain a list of images from 2010 to 2018 between August and November

In [5]:
# This returns a list of all images ID
Rainfall=ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filterDate("2018-08-01","2018-11-30")\
.sort("system:time_start").aggregate_array("system:index").getInfo() # Return a list of image ID

In [6]:
container=[]
years=list(range(2010,2019))
for eachyear in years:
    Rainfall=ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filterDate("{0}-08-01".format(eachyear),"{0}-11-30".format(eachyear))\
.sort("system:time_start").aggregate_array("system:index").getInfo()
    for eachimage in Rainfall:
        container.append(eachimage)

### 2.1.2. Building an ImageCollection from above container (precipitation data)

In [7]:
list_image=[]

for eachimage in container:
    image=ee.Image('UCSB-CHG/CHIRPS/PENTAD/{0}'.format(eachimage))
    list_image.append(image)
Rainfall_Collection=ee.ImageCollection(list_image)
print(Rainfall_Collection.size().getInfo())

216


In [13]:
# Obtain the median rainfall over the period
median_rainfall=Rainfall_Collection.median()
# Obtain the max rainfall over the period
max_rainfall=Rainfall_Collection.max()
# Obtain the min rainfall over the period
min_rainfall=Rainfall_Collection.min()
# Display max rainfall
vis_params = {
  'min': 50,
  'max': 200,
  'palette': ['#fdff19', '#14ff17', '#ff1308', '#041fff', '#10faff']}
Map.addLayer(max_rainfall,vis_params,"Max_Rainfall")

### 1.2.3. Export rainfall to Google Drive

In [9]:
HN1=HN.geometry()
path=os.path.join(r"F:\Research\Research_Cooperation\ILRI_Mosquito_Mapping\Mosquito\Python_Mosquito","Max_Rain.tif")
emap.ee_export_image(max_rainfall, filename=path,scale=30,region=HN1,file_per_band=False)

Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'F:\\Research\\Research_Cooperation\\ILRI_Mosquito_Mapping\\Mosquito\\Python_Mosquito\\Max_Rain.zip'


In [23]:
task = ee.batch.Export.image.toDrive(image=max_rainfall,  # an ee.Image object.
                                     region=HN1,  # an ee.Geometry object.
                                     description='Max_Rain',
                                     folder='Mosquito',
                                     fileNamePrefix='Max_Rain',
                                     scale=30)
task.start() # Nothing happens

# 2. Temperature